# Training the model to run on Lichess data
## Some pre-requisites if running on Google Collab
If not running on Google collab do not run these next two cells!

In [ ]:
# Install the only dependency not available from collab directly
!pip install chess

# Get imported files from repo
!git clone -b rl-setup https://github.com/owenjaques/chessbot.git
!mv chessbot chessbot-repo
!mv chessbot-repo/neural_networks/chessbot .
!rm chessbot-repo -r

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
weights_directory = '/content/gdrive/MyDrive/chessbot_weights/'
print(f'Saving weights to {weights_directory}')

## If not running on Google Collab
Set the weights directory variable to wherever you would like data saved.

In [ ]:
weights_directory = 'your directory here'

## Get the data
This compression format is really nice, so you can cancel this cell whenever you want and all the games that were downloaded will be maintained. In my experience 300Mb gets well over 100,000 games.

In [ ]:
!wget https://database.lichess.org/standard/lichess_db_standard_rated_2023-02.pgn.zst

## Decompress the Data

In [ ]:
!apt install zstd
!pzstd -d lichess_db_standard_rated_2023-02.pgn.zst

## Play the games from the data
For this section we want to play the games so we can translate them into labelled model inputs which we can train on. 

In [ ]:
import chess
import chess.pgn
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from collections import deque
from pathlib import Path

In [ ]:
MAX_GAMES = 25000
CHECKPOINT_N_GAMES = 2500

In [ ]:
from chessbot.model_input import ModelInput

X_all = deque()
y_expectation_all = deque()
y_score_all = deque()

with open('lichess_db_standard_rated_2023-02.pgn') as pgn:
    game_count = 0
    game = chess.pgn.read_game(pgn)

    while game is not None and game_count < MAX_GAMES:
        result = game.headers['Result']
        
        # Only train on game played to completion, that were not draws, and that have evaluations
        next_node = game.next()
        if game.headers['Termination'] == 'Normal' and result in ['1-0', '0-1'] and next_node and next_node.eval() != None:
            print(f'\rProcessing game {game_count}/{MAX_GAMES}', end='')
            
            X = []
            y_expectation = []
            y_score = []

            try:
                # Generate the data from the game
                board = game.board()
                for node in game.mainline():
                    board.push(node.move)
                    X.append(ModelInput(board).get_input())

                    node_eval = node.eval()
                    y_expectation.append(node_eval.wdl().white().expectation())
                    y_score.append(node_eval.white().score(mate_score=10000))

                X_all.extend(X)
                y_expectation_all.extend(y_expectation)
                y_score_all.extend(y_score)

                game_count += 1
            except:
                # There are a lot of reasons an exception could be thrown here, mostly stemming from bad data being parsed
                # from the pgn file. We just ignore these games and move on.
                pass

            # Checkpoint current game data
            if game_count % CHECKPOINT_N_GAMES == 0 and game_count != 0:
                X = np.array(X_all)
                y_expectation = np.array(y_expectation_all)
                y_score = np.array(y_score_all)
                
                X_all.clear()
                y_expectation_all.clear()
                y_score_all.clear()
                
                path = f'{weights_directory}{game_count}_games_data.npz'
                if not Path(path).is_file():
                    np.savez_compressed(path, X=X, y_expectation=y_expectation, y_score=y_score)

        # Get the next game
        game = chess.pgn.read_game(pgn)

### Load data from previous step
In this cell you can choose which labels you want to load from the stored data. Current options are the score, `y_score`, and the wdl expectation, `y_expectation`.

In [ ]:
y_to_load = 'y_score'

Running this cell will load the games from the last step that were saved to disk. Expect to see high RAM usage.

In [ ]:
X = None
y = None

n_games = CHECKPOINT_N_GAMES
data_file = f'{weights_directory}{n_games}_games_data.npz'
while Path(data_file).is_file() and n_games <= MAX_GAMES:
    print(f'\rLoading data from {data_file}', end='')
    data = np.load(data_file)

    if n_games == CHECKPOINT_N_GAMES:
        X = data['X']
        y = data[y_to_load]
    else:
        X = np.concatenate([X, data['X']])
        y = np.concatenate([y, data[y_to_load]])

    n_games += CHECKPOINT_N_GAMES
    data_file = f'{weights_directory}{n_games}_games_data.npz'

X = np.array(X)
y = np.array(y)

print(f'\nNumber of samples: {X.shape[0]}, number of labels: {y.shape[0]}')

If for whatever reason you only want to load one of the checkpoint files, run this cell instead of the above one (and change `data_file` to point to where the checkpoint file you want to load is).

In [ ]:
data_file = f'{weights_directory}2500_games_data.npz'

data = np.load(data_file)
X = data['X']
y = data[y_to_load]

### Plot the distribution of the training set

In [ ]:
n_bins = 100

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

ax[0].hist(y, bins=n_bins)
ax[0].set_title('Distribution of training labels')
ax[0].set_xlabel('label (y)')
ax[0].set_ylabel('no. of occurences in dataset')

y_tmp = y[y < 2500]
y_tmp = y_tmp[y_tmp > -2500]
ax[1].hist(y_tmp, bins=n_bins)
ax[1].set_title('Distribution of training labels (zoomed in)')
ax[1].set_xlabel('label (y)')
ax[1].set_ylabel('no. of occurences in dataset')

plt.show()

#### Experiment to balance training set
If you look at the first histogram in this experiment you will see that the training set is unbalanced, this cell aims to balance the training set by undersampling it.

In [ ]:
from sklearn.utils import shuffle

# Note this does not currently work

X, y = shuffle(X, y)

bins = np.linspace(0, 1, n_bins)
binned_y_indices = np.digitize(y, bins)
unique_bins, counts = np.unique(binned_y_indices, return_counts=True)
n_to_sample = np.min(counts)
n_sampled = counts.astype(int)

mask = np.full(y.shape[0], False, dtype=bool)
for i in range(y.shape[0]):
    if n_sampled[binned_y_indices[i] - 1] < n_to_sample:
        mask[i] = True
        n_sampled[binned_y_indices[i] - 1] += 1

X_resampled = X[mask]
y_resampled = y[mask]

fig, ax = plt.subplots()
ax.hist(y_resampled, bins=n_bins)
ax.set_title('Distribution of training labels (undersampled)')
ax.set_xlabel('label (y)')
ax.set_ylabel('no. of occurences in dataset')
plt.show()

X = X_resampled
y = y_resampled

### Normalising the dataset (where y is the score)

In [ ]:
# Threshold to remove outliers and increase distribution of data
y[y > 1000] = 1000
y[y < -1000] = -1000

# Normalise in the range [0, 1]
y = (y - np.min(y)) / (np.max(y) - np.min(y))

fig, ax = plt.subplots()
ax.hist(y, bins=n_bins)
ax.set_title('Distribution of training labels (normalised)')
ax.set_xlabel('label (y)')
ax.set_ylabel('no. of occurences in dataset')
plt.show()

## Our model
Set up your model being used here.

In [ ]:
model = keras.Sequential([
	keras.layers.Dense(512, activation='relu'),
	keras.layers.Dense(512, activation='relu'),
	keras.layers.Dense(512, activation='relu'),
	keras.layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss='mse',
	metrics=[keras.metrics.MeanAbsoluteError()]
)

## Training the model
This next cell trains the model on the training data, then saves it to disk. Note multiple calls to this cell have crashed the notebook before due to high RAM usages.

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=3, verbose=1)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.00001, verbose=1)

model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, shuffle=True, callbacks=[early_stopping, reduce_lr])
model.save(f'{weights_directory}lichess_trained_model')

## Why not play a game after all that training?

In [ ]:
import time
from IPython.display import clear_output
from chessbot.chessbot import ChessBot

def play_game(model, exploration_rate=0.0, should_visualise=False):
	white = ChessBot(model, chess.WHITE, exploration_rate)
	#black = ChessBot(model, chess.BLACK, exploration_rate)

	board = chess.Board()

	if should_visualise:
		display(board)

	while not board.is_game_over(claim_draw=True):
		board.push(chess.Move.from_uci(input()) if board.turn == chess.BLACK else white.move(board))

		if should_visualise:
			clear_output(wait=True)
			display(board)
			time.sleep(0.5)

	return board.outcome(claim_draw=True).result()
 
play_game(model, should_visualise=True)